## EXPLORATION 9 

### 데이터 셋

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [5]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### EDA

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### ADD RATINGS

In [10]:
my_favorite = ['Terminator, The (1984)' , 'Forrest Gump (1994)' ,'Matrix, The (1999)' ,'American Beauty (1999)' ,'Jurassic Park (1993)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)
ratings.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
352,6041,356,5
476,6041,480,5
1220,6041,1240,5
2502,6041,2571,5
2789,6041,2858,5


In [11]:
favorite_movie_id

,movie_id,title,genre
352,356,Forrest Gump (1994),Comedy|Romance|War
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2789,2858,American Beauty (1999),Comedy|Drama


In [12]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


### CSR matrix

In [13]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
352,6041,356,5
476,6041,480,5
1220,6041,1240,5
2502,6041,2571,5


In [14]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

In [15]:
num_user

6040

In [16]:
num_movie

3628

In [17]:
ratings.user_id

0          1
1          1
2          1
3          1
4          1
        ... 
352     6041
476     6041
1220    6041
2502    6041
2789    6041
Name: user_id, Length: 836483, dtype: int64

In [18]:
ratings.movie_id

0       1193
1        661
2        914
3       3408
4       2355
        ... 
352      356
476      480
1220    1240
2502    2571
2789    2858
Name: movie_id, Length: 836483, dtype: int64

In [19]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### Train

In [20]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [24]:
my_vector, matrix_vector = als_model.user_factors[6041], als_model.item_factors[2571]

In [25]:
my_vector

array([ 0.4072757 ,  0.22074312, -1.1567761 , -1.376396  ,  0.8865208 ,
       -0.12359031,  0.0454787 ,  0.05849811,  0.4203616 ,  0.10955789,
        0.49489373, -0.57332176, -0.19766816,  0.44555393, -0.03865648,
        1.0636348 ,  0.5055285 , -0.84486586,  0.31341055,  0.08224048,
       -1.0946454 , -0.03440435, -0.7294693 ,  0.00823712,  0.26795986,
        0.52715886, -0.59133816, -0.78150123, -0.00160705,  0.13637236,
        0.2651085 , -0.4326762 , -0.49801034, -0.54589015,  0.6049305 ,
       -0.5327398 , -0.35223246,  0.50188357,  0.31920546,  0.00698727,
        0.46402094, -0.25283787,  0.42511666,  0.79724526, -0.10382189,
       -0.7168661 , -0.06471165,  0.8491044 , -0.59951293,  0.72391856,
        0.08422125,  0.31263784, -0.1557008 ,  0.11015803,  0.08347145,
        0.5978939 ,  0.11112355, -0.3836904 ,  0.56529176,  0.02011552,
       -0.2710444 , -0.34792337, -0.2971958 ,  0.12358961, -0.800646  ,
       -0.14008991,  0.7975431 ,  0.39161417, -0.01643161, -0.34

In [26]:
matrix_vector

array([-9.86660551e-03, -3.57150449e-03, -2.25781798e-02, -3.82233933e-02,
        3.35695706e-02,  1.12236189e-02,  9.31198522e-03,  7.02521019e-03,
        1.76684819e-02,  1.08769657e-02,  2.53455210e-02,  5.08439401e-03,
        7.43988110e-03,  1.11678401e-02,  6.28736103e-03,  3.11842449e-02,
        2.57836189e-02, -4.17386997e-04,  6.53644698e-03,  1.25892262e-03,
       -2.48005800e-02, -5.06285531e-03,  9.19354241e-03, -5.81666408e-03,
        2.40139347e-02,  2.25333758e-02, -1.28795775e-02, -1.46252895e-02,
       -1.59928687e-02,  1.85448658e-02,  8.90398864e-03, -9.59436595e-03,
        1.59464069e-02, -5.92687121e-03,  2.69279536e-02, -1.70090869e-02,
       -1.25558432e-02,  2.47061811e-03,  5.13944402e-03,  7.71607645e-03,
        2.00609788e-02, -1.05826342e-02,  2.46730614e-02,  3.74994315e-02,
        1.08835725e-02,  1.66422334e-02,  3.46072912e-02,  3.41063663e-02,
       -6.27292693e-03,  2.36284304e-02,  1.60006117e-02, -8.03477364e-04,
        1.81828141e-02,  

In [28]:
np.dot(my_vector, matrix_vector)

0.60287774

In [29]:
toystory_vector = als_model.item_factors[1]
np.dot(my_vector, toystory_vector)

0.18603422

### Similar Movie

In [30]:
favorite_movie = 'Matrix, The (1999)'
movie_id = movies[movies['title']=='Matrix, The (1999)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(2571, 0.9999999),
 (589, 0.8093728),
 (2916, 0.6857466),
 (457, 0.6205933),
 (1240, 0.589003),
 (1527, 0.5597451),
 (1573, 0.5292174),
 (480, 0.52497876),
 (1580, 0.4598921),
 (1610, 0.45085946),
 (32, 0.4440201),
 (260, 0.40063706),
 (1214, 0.39571533),
 (2028, 0.39096263),
 (1538, 0.38342363)]

In [31]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
31,32,Twelve Monkeys (1995),Drama|Sci-Fi
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,457,"Fugitive, The (1993)",Action|Thriller
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
1491,1527,"Fifth Element, The (1997)",Action|Sci-Fi
1500,1538,"Second Jungle Book: Mowgli & Baloo, The (1997)",Adventure|Children's
1533,1573,Face/Off (1997),Action|Sci-Fi|Thriller


In [32]:
user = 6041
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.7196867),
 (1580, 0.4881497),
 (1210, 0.47682336),
 (110, 0.40292728),
 (2916, 0.39378202),
 (1196, 0.36922795),
 (260, 0.35262668),
 (2762, 0.34212327),
 (1270, 0.32874224),
 (593, 0.29432195),
 (457, 0.2796699),
 (2028, 0.27551347),
 (2628, 0.27520218),
 (1097, 0.26353282),
 (1200, 0.25993103),
 (2997, 0.24951428),
 (1265, 0.24079315),
 (1214, 0.22427404),
 (377, 0.20662577),
 (3101, 0.20504552)]

In [33]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
373,377,Speed (1994),Action|Romance|Thriller
453,457,"Fugitive, The (1993)",Action|Thriller
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
